In [1]:
##Importing pyodbc module
import pandas as  pd
import pyodbc as odbc

#Lib Graph
import plotly.figure_factory as ff
import plotly.offline as offline
import plotly.graph_objs as go
import numpy as np
offline.init_notebook_mode()
 
##connecting to your database through ODBC
cnxn = odbc.connect('DSN=250-serv;UID=sa;PWD=pass1234', autocommit=True)
cursor = cnxn.cursor()

##connecting to your SECOND database through ODBC
server = odbc.connect('DSN=196-serv;UID=support;PWD=support', autocommit=True)
cursor2 = server.cursor() 

#overperiod
year = "'2019-"
month = "05-"

#week days -----------
s1start = " 06:00:00'"
s1end = " 13:30:00'"
s2start = " 13:30:00'"
s2end = " 21:00:00'"
s3start = " 21:00:00'"
s3end = " 06:00:00'"

#sabtu ----------------
ss1start = " 06:00:00'"
ss1end = " 11:00:00'"
ss2start = " 11:00:00'"
ss2end = " 16:00:00'"
ss3start = " 16:00:00'"
ss3end = " 21:00:00'"

#conv name -----------
conv = "'E011-V'"
conv2 = "'Conv-V-Omron'"

#date and shift ------

if month == "04-" or month == "06-" or month == "09-"or month == "11-":
    maxtgl = 30
maxshift = 4
maxtgl = 31

#dictionaries --------
dictshift = {}
dbvarab = {}
dbvarom = {}
dErr_ab = {}
dErr_om = {}

#Seven_Segment data Error
dmcount = np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
                   ,0,4100,3102,3709,3467,0,0,0,0,0,0,0,0,0,0,0,0,0,0
                   ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
                   ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
                   ,0,0,0,0,0,0,0
                   ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
                   ,0,0,0,0,0,0,0
                   ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
                   ,0,0,0,0,0,0,0
                   ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])

#local var zero ------
sev_seg = 0


#Percentages______
Jumlah_SCHdev  = 0
Jumlah_ABdev   = 0
Jumlah_MCount  = 0

#Strcuk Main DataFrame
Allsum = pd.DataFrame({'Shift': [], 'Omron': [], 'Allen_Bradley': [], 'Manual_Count': [], 'Deviasi_Omron': [],
                       'Deviasi_Allen_Bradley': [], 'Deviasi_AB_OMR': [],
                       'Err_Data_AB': [], 'Err_Data_OMR': [],})


# Main Looping code (while (1))________________________________________________________________________________________
for tgl in range (1,maxtgl):
    for shift in range (1,maxshift):
        tanggal = "0"+str(tgl)
        tanggal_n= "0"+str(tgl)
        errab = 0
        errom = 0
        bedaab = 0 
        bedaom = 0
        
        
        if shift == 3 :
            tanggal_n = "0"+str(tgl+1)
            if tgl >= 9:
                tanggal_n = str(tgl+1)
                    
        if tgl > 9 :
            tanggal   = str(tgl)
            tanggal_n = str(tgl)
            
        
        s_shift = "s"+str(shift)+"start"
        e_shift = "s"+str(shift)+"end"
        ss_shift = "ss"+str(shift)+"start"
        se_shift = "ss"+str(shift)+"end"
        
        if shift == 3 and tgl == maxtgl:
            tanggal = "1"
            month   = "06-"
                    
        if tgl != 5 and tgl != 12 and tgl != 19 and tgl != 26:
            dbvarab["s"+str(shift)+tanggal+"ab"]= pd.read_sql('SELECT *  FROM mlog2 where tgljam2 between '+year+month+tanggal+eval(s_shift)+' and ' +year+month+tanggal_n+eval(e_shift)+'and namamesin ='+conv+' ORDER BY nobatch asc',server)
            dbvarom["s"+str(shift)+tanggal+"om"]= pd.read_sql('SELECT *  FROM Table_SiteTest where tgljam between '+year+month+tanggal+eval(s_shift)+' and ' +year+month+tanggal_n+eval(e_shift)+'and namamesin ='+conv2+' ORDER BY nobatch asc',cnxn)
                        
            if tgl == 4 or tgl == 11 or tgl == 18 or tgl == 25:
                dbvarab["s"+str(shift)+tanggal+"ab"] = pd.read_sql('SELECT *  FROM mlog2 where tgljam2 between ' +year+month+tanggal+eval(ss_shift)+ ' and ' +year+month+tanggal_n+eval(se_shift)+' and namamesin ='+conv+' ORDER BY nobatch asc',server)
                dbvarom["s"+str(shift)+tanggal+"om"] = pd.read_sql('SELECT *  FROM Table_SiteTest where tgljam between ' +year+month+tanggal+eval(ss_shift)+ ' and ' +year+month+tanggal_n+eval(se_shift)+' and namamesin ='+conv2+' ORDER BY nobatch asc',cnxn)
            
            paramab = dbvarab["s"+str(shift)+tanggal+"ab"]
            
            for i in range (paramab.namamesin.count()-1):
                if paramab.index[i] - (paramab.nobatch.iloc[i] + bedaab) !=0 :
                    bedaab = paramab.index[i] - paramab.nobatch.iloc[i]
                    if i != 0:
                        errab += 1
                    #dErr_ab["s"+str(shift)+tanggal+"ab"]= errab

            paramom = dbvarom["s"+str(shift)+tanggal+"om"]
            
            for j in range (paramom.namamesin.count()-1):
                if paramom.index[j] - (paramom.nobatch.iloc[j] + bedaom) !=0 :
                    bedaom = paramom.index[j] - paramom.nobatch.iloc[j]
                    if j != 0:
                        errom += 1 
                    #dErr_om["s"+str(shift)+tanggal+"om"]= errom   
                        
            Allsum = Allsum.append({ 'Shift'          : tanggal+"May-Shift"+str(shift), 
                             'Omron'                  : dbvarom["s"+str(shift)+tanggal+"om"].namamesin.count(), 
                             'Allen_Bradley'          : dbvarab["s"+str(shift)+tanggal+"ab"].namamesin.count(),
                             'Manual_Count'           : dmcount[sev_seg], 
                             'Deviasi_Omron'          : abs(dmcount[sev_seg] - dbvarom["s"+str(shift)+tanggal+"om"].namamesin.count()),     
                             'Deviasi_Allen_Bradley'  : abs(dmcount[sev_seg] - dbvarab["s"+str(shift)+tanggal+"ab"].namamesin.count()), 
                             'Deviasi_AB_OMR'         : abs(dbvarab["s"+str(shift)+tanggal+"ab"].namamesin.count() - dbvarom["s"+str(shift)+tanggal+"om"].namamesin.count()),
                             'Err_Data_AB'            : errab,     
                             'Err_Data_OMR'           : errom}, 
                           ignore_index=True)
            sev_seg += 1
            
####Visualisation
trace1 = go.Bar(
                    x=Allsum.Shift,
                    y=Allsum.Omron ,    
                    marker=dict(color='#ffcdd2'),
                    name='Omron'
                )
trace2 = go.Bar(
                    x=Allsum.Shift,
                    y=Allsum.Allen_Bradley,
                    marker=dict(color='#A2D5F2'),
                    name='Allen_Bradley'
                )
trace3 = go.Line(
                    x=Allsum.Shift,
                    y=Allsum.Manual_Count,
                    marker=dict(color='#28193a'),
                    name='MCount'
                )
data = [trace1,trace2,trace3]

layout = go.Layout(
    
                    xaxis=dict(title='Time'),
                    yaxis=dict(title='Value'),
                    title = "Comparing ALL"
                  )  
fig = go.Figure(data=data, layout=layout)

#### Error Visualisation
grapherr1 = go.Bar(
                    x=Allsum.Shift,
                    y=Allsum.Err_Data_OMR,    
                    marker=dict(color='#9df2cb'),
                    name='Error_Omron'
                )
grapherr2 = go.Bar(
                    x=Allsum.Shift,
                    y=Allsum.Err_Data_AB,
                    marker=dict(color='#f1d99c'),
                    name='Error_Allen_Bradley'
                )
grapherr3 = go.Scatter (
                    x=Allsum.Shift,
                    y=Allsum.Deviasi_AB_OMR,
                    marker=dict(color='#d6ef15'),
                    name='Deviasi_AB_OMR',
                    mode = 'lines'
                )
grapherr4 = go.Line(
                    x=Allsum.Shift,
                    y=Allsum.Deviasi_Omron,
                    marker=dict(color='#14ef7d'),
                    name='Deviasi_Omron_Mcount'
                )
grapherr5 = go.Line(
                    x=Allsum.Shift,
                    y=Allsum.Deviasi_Allen_Bradley,
                    marker=dict(color='#9b14ef'),
                    name='Deviasi_Allen_Bradley_MCount'
                )

        

dataerror = [grapherr1,grapherr2,grapherr3,grapherr4,grapherr5]

layout = go.Layout(    
                    xaxis=dict(title='Time'),
                    yaxis=dict(title='Value'),
                    title = "Error"
                  )  
fig = go.Figure(data=dataerror , layout=layout)


for i in range (Allsum.Shift.count() - 1):
    if i != 5 or 6:
        Jumlah_SCHdev = Jumlah_SCHdev + abs(Allsum.Omron.iloc[i] - Allsum.Manual_Count.iloc[i])
        Jumlah_ABdev  = Jumlah_ABdev + abs(Allsum.Allen_Bradley.iloc[i]  - Allsum.Manual_Count.iloc[i])
        Jumlah_MCount = Jumlah_MCount + Allsum.Manual_Count.iloc[i]
     
    
dataPercent = [['SCH', (1-(Jumlah_SCHdev /Jumlah_MCount))*100],['AB', (1-(Jumlah_ABdev /Jumlah_MCount))*100], ] 
Persentase= pd.DataFrame(dataPercent, columns = ['Controller', 'Percentages'])


#---------------------------------------------------------------------------------------------------------------- 
labels = ['Allen_bradley','Omron']
values = [Allsum.Err_Data_AB.sum(),Allsum.Err_Data_OMR.sum()]
colors = ['#FEBFB3', '#E1396C']

pie = go.Pie(labels=labels, values=values,
               hoverinfo='label+percent', textinfo='value', 
               textfont=dict(size=20),
               marker=dict(colors=colors, 
                           line=dict(color='#000000', width=2)))
#--------------------------------------------------------------------------------------------------------------------

AttributeError: module 'numpy' has no attribute '__version__'

In [ ]:
offline.iplot(data, filename= "Comparison-All")
Allsum.iloc[27:58]

In [ ]:
offline.iplot(dataerror, filename= "Comparison-All")

## Check your detail data here >> param as input

In [ ]:
paramsc = dbvarom['s113om']
beda = 0
errsc = 0 
for i in range (paramsc.namamesin.count() -1):
     if paramsc.index[i] - (paramsc.nobatch.iloc[i] + beda)  !=0 :
            #print (i)
            print(paramsc.nobatch.loc[i-2:i+2])
            beda = paramsc.index[i] - paramsc.nobatch.iloc[i]
            if i != 0 :
                errsc += 1
            print (errsc)

## Continued Detail >> dbvarname['shift&date'].iloc[range]

## Pie in Completness Data

In [ ]:
offline.iplot([pie], filename='basic_pie_chart')